# Associating Slogfiles with Nodes, Tasks

### Preface: PyData tools

In [1]:
import pandas as pd
dict(pandas=pd.__version__)

{'pandas': '1.3.2'}

## Task Portal Export

Participants use the https://validateagoric.knack.com/ portal to submit their tasks for review.

In [54]:
!ls -l portal-export/submittedtasks.csv

-rw-r--r-- 1 jupyter jupyter 11024832 Aug 28 13:58 portal-export/submittedtasks.csv


In [6]:
tasksub = pd.read_csv('portal-export/submittedtasks.csv',
                      parse_dates=['Last Date Updated'])
tasksub.tail(3)

,Event,Task,Discord ID,Moniker,Developer,Submission Link,Task Type,Points,Status,Verified,TaskBoardID,Last Date Updated
4624,Stress Test,Restart your validator twice during agorictest-15,luizapplima#8073,Papaya Node,Luiza Do Prado Lima,NaN,Network Task,400,Completed,Not accepted,2887,2021-06-28 10:36:00
4625,Stress Test,Restart your validator twice during agorictest-15,CreATivE8536#5039,isavazh,Mokhamed Savazh,Moniker - isavazh<br /><br />agorictest-13 - 2...,Network Task,400,Completed,Not accepted,2835,2021-06-27 08:06:00
4626,Stress Test,Restart your validator twice during agorictest-15,Edouard Lavidalle | Stakin#2231,Stakin,Edouard Lavidalle,NaN,Network Task,400,Completed,Not accepted,2880,2021-06-28 06:41:00


In [8]:
phase45start = '2021-08-15'
tasksub[tasksub['Last Date Updated'] >= phase45start].groupby(['Event', 'Task'])[['TaskBoardID']].count()

TaskBoardID
Event     Task                                                           
Metering  AMM Trade Part 1: Make a trade on the AMM and g...           82
          AMM Trade Part 2: Find the crank number of the ...           70
          Capture and submit a slog file                               95
          Conduct performance analysis                                 50
          Create a contract that charges fees to users                  2
          Create and submit gentx - Metering                          150
          Launch and maintain Agoric's load generator nod...          100
          Maintain uptime during phase!                                89
          Restart your validator at least three times at ...          112
          Start your validator as part of the genesis cer...          125
          Tweet a GIF about about phase 4.5                            88

## Slogfile tasks submitted

In [18]:
taskslog = tasksub[tasksub['Task'] == 'Capture and submit a slog file'].drop(columns=['Event', 'Task', 'Points', 'Task Type']).set_index('TaskBoardID')
print(len(taskslog))
taskslog.tail()

95


,Discord ID,Moniker,Developer,Submission Link,Status,Verified,Last Date Updated
TaskBoardID,,,,,,,
4516,asif#1713,almuezza,Asif Jamadar,Name: 2021-08-26T20:55:54.867Z-asif#1713.slog....,Completed,In review,2021-08-26 16:33:00
4635,chris-remus#4082,Chainflow,Chris Remus,slogfile available here -<br /><br />https://c...,Completed,In review,2021-08-27 20:49:00
4622,svAbhishek | OmniFlix Network#1613,OmniFlix Network,Sistla Venkata Abhishek,name : 2021-08-27T22:10:31.625Z-svAbhishek | O...,Completed,In review,2021-08-27 18:25:00
4623,Coolex#3021,coolex,Anatolii Shardubin,uploaded via https://submit.agoric.app,Completed,In review,2021-08-27 18:48:00
4641,mr_vavilon#6450,mrvavilon,Egor Malakhov,Completted<br />https://drive.google.com/file/...,Completed,In review,2021-08-28 01:30:00


## Slogfiles Collected

The https://submit.agoric.app tool authenticated participants using Discord OAuth
and stored their submissions in the `slogfile-upload-5` bucket in Google Cloud Storage.

In [86]:
up5 = !gsutil ls -l gs://slogfile-upload-5/

In [87]:
def pathparts(row):
    filename = row.path.split('/')[-1]
    noext = filename.replace('.slog.gz', '')
    fresh, discordID = noext.split('Z-', 1)
    return dict(filename=filename, fresh=fresh, discordID=discordID)

sf5 = pd.DataFrame.from_records(
    [dict(size=int(size), modified=modified, path=path) for line in up5 if 'Z' in line
     for (size, modified, path) in [line.strip().split('  ', 2)]])
sf5 = pd.concat([sf5, sf5[['path']].apply(pathparts, axis=1, result_type='expand')], axis=1)
sf5 = sf5.set_index('filename').sort_index()
sf5 = sf5.assign(modified=pd.to_datetime(sf5.modified),
                 fresh=pd.to_datetime(sf5.fresh))
print(sf5.dtypes)
sf5.drop(columns=['path']).sort_values('discordID')

size                       int64
modified     datetime64[ns, UTC]
path                      object
fresh             datetime64[ns]
discordID                 object
dtype: object


,size,modified,fresh,discordID
filename,,,,
2021-08-27T07:52:31.251Z-4andr#6771.slog.gz,1720723075,2021-08-27 08:36:06+00:00,2021-08-27 07:52:31.251,4andr#6771
2021-08-27T08:32:23.278Z-<-ThomaS->#0485.slog.gz,1692779239,2021-08-27 08:35:21+00:00,2021-08-27 08:32:23.278,<-ThomaS->#0485
2021-08-26T13:07:51.018Z-Adorid | adorid.xyz#1293.slog.gz,1646890698,2021-08-26 13:09:13+00:00,2021-08-26 13:07:51.018,Adorid | adorid.xyz#1293
2021-08-26T19:34:10.967Z-Alex Turetskiy#5973.slog.gz,68125044,2021-08-26 19:36:49+00:00,2021-08-26 19:34:10.967,Alex Turetskiy#5973
2021-08-27T14:45:02.111Z-Aliaksandr#3647.slog.gz,773584703,2021-08-27 14:50:04+00:00,2021-08-27 14:45:02.111,Aliaksandr#3647
...,...,...,...,...
2021-08-27T12:00:56.385Z-mirxl#0530.slog.gz,3925214742,2021-08-27 13:57:03+00:00,2021-08-27 12:00:56.385,mirxl#0530
2021-08-28T05:34:31.973Z-paddyson#5479.slog.gz,1955047404,2021-08-28 05:40:44+00:00,2021-08-28 05:34:31.973,paddyson#5479
2021-08-27T04:12:28.528Z-pfrp#1964.slog.gz,964869285,2021-08-27 04:15:39+00:00,2021-08-27 04:12:28.528,pfrp#1964


### Duplicate, empty uploads

These were moved to `dup-test`.

In [81]:
sf5[sf5.duplicated(['discordID', 'size'], keep=False)]

,size,modified,path,fresh,discordID
filename,,,,,


In [74]:
sfold = sf5[sf5.duplicated(['discordID', 'size'], keep='last')]
sfold.path

filename
2021-08-25T21:05:56.611Z-kodvas#8552.slog.gz                    gs://slogfile-upload-5/2021-08-25T21:05:56.611...
2021-08-26T01:21:53.208Z-iicc | stakely.io#4715.slog.gz         gs://slogfile-upload-5/2021-08-26T01:21:53.208...
2021-08-26T02:05:14.547Z-pfrp#1964.slog.gz                      gs://slogfile-upload-5/2021-08-26T02:05:14.547...
2021-08-26T05:03:03.952Z-Adorid | adorid.xyz#1293.slog.gz       gs://slogfile-upload-5/2021-08-26T05:03:03.952...
2021-08-26T08:40:06.517Z-Legio#4947.slog.gz                     gs://slogfile-upload-5/2021-08-26T08:40:06.517...
2021-08-26T10:35:13.450Z-<-ThomaS->#0485.slog.gz                gs://slogfile-upload-5/2021-08-26T10:35:13.450...
2021-08-26T11:16:43.069Z-Vitalii#9679.slog.gz                   gs://slogfile-upload-5/2021-08-26T11:16:43.069...
2021-08-26T12:47:19.872Z-YH | StakeWithUs#6077.slog.gz          gs://slogfile-upload-5/2021-08-26T12:47:19.872...
2021-08-26T17:54:12.019Z-Serhio911#2565.slog.gz                 gs://slogfile-u

In [73]:
from slogdata import CLI

def _cli(bin):
    from subprocess import run, Popen
    return CLI(bin, run, Popen)

gsutil = _cli('gsutil')

In [75]:
sfold.path[0]

'gs://slogfile-upload-5/2021-08-25T21:05:56.611Z-kodvas#8552.slog.gz'

In [76]:
gsutil.run('rm', sfold.path[0])

CompletedProcess(args=['gsutil', 'rm', 'gs://slogfile-upload-5/2021-08-25T21:05:56.611Z-kodvas#8552.slog.gz'], returncode=0, stdout=b'', stderr=b'Removing gs://slogfile-upload-5/2021-08-25T21:05:56.611Z-kodvas#8552.slog.gz...\n/ [1 objects]                                                                   \r\nOperation completed over 1 objects.                                              \n')

In [82]:
gsutil.run('mv', *sfold.path[1:], 'gs://slogfile-upload-5/dup-test/');

## Connecting Slogfiles with Task Submissions

In [126]:
sf5match = pd.merge(taskslog.reset_index(), sf5.reset_index(),
         left_on='Discord ID', right_on='discordID', how='left'
                   ).set_index('TaskBoardID').drop(columns=['Developer', 'Status', 'path', 'fresh', 'discordID', 'modified'])

sf5match.loc[~sf5match.filename.isnull(), 'Verified'] = 'Accepted'

sf5match = sf5match.sort_values(['Verified', 'Discord ID', 'filename'])

def move_col(df, col, pos):
    x = df[col]
    df = df.drop(columns=[col])
    df.insert(pos, col, x)
    return df

sf5match = move_col(sf5match, 'Verified', 1)
sf5match.head()

,Discord ID,Verified,Moniker,Submission Link,Last Date Updated,filename,size
TaskBoardID,,,,,,,
4566,4andr#6771,Accepted,4andr,provided to you via https://submit.agoric.app<...,2021-08-27 04:52:00,2021-08-27T07:52:31.251Z-4andr#6771.slog.gz,1.720723e+09
4360,Adorid | adorid.xyz#1293,Accepted,Adorid,Uploaded,2021-08-26 09:15:00,2021-08-26T13:07:51.018Z-Adorid | adorid.xyz#1...,1.646891e+09
4227,Alex Turetskiy#5973,Accepted,TuretskiyV,TuretskiyV-agorictest17-chain:<br />https://dr...,2021-08-25 19:30:00,2021-08-26T19:34:10.967Z-Alex Turetskiy#5973.s...,6.812504e+07
4578,Aliaksandr#3647,Accepted,Coolcat,CoolCat<br /><br />https://drive.google.com/fi...,2021-08-27 09:52:00,2021-08-27T14:45:02.111Z-Aliaksandr#3647.slog.gz,7.735847e+08
4499,Anna1242#2262,Accepted,dreamstaker,"I uploaded it on submit.agoric.app, but due to...",2021-08-26 15:34:00,2021-08-26T19:12:39.249Z-Anna1242#2262.slog.gz,1.795795e+09


In [117]:
sf5match.groupby('Verified')[['Discord ID']].aggregate('nunique')

,Discord ID
Verified,
Accepted,53
In review,41


In [127]:
sf5match[sf5match.Verified != 'Accepted']

,Discord ID,Verified,Moniker,Submission Link,Last Date Updated,filename,size
TaskBoardID,,,,,,,
4336,#0485,In review,AlphaBravo,Complete,2021-08-26 08:05:00,NaN,NaN
4610,13610152128#5923,In review,nodestradamus,nodestradamus-agorictest17-chain.slog.gz,2021-08-27 15:41:00,NaN,NaN
4629,86b#6313,In review,oraclegen,https://agoric.s3.us-east-2.amazonaws.com/orac...,2021-08-27 19:48:00,NaN,NaN
4389,@calin_stakeborg,In review,Stakeborg,Slog file uploaded on https://submit.agoric.ap...,2021-08-26 10:52:00,NaN,NaN
4504,AndreyRudi#2065,In review,rudolfhe-55,https://storage.nodes.guru/rudolfhe-55-agorict...,2021-08-26 15:51:00,NaN,NaN
4398,ColinkaMalinka#1565,In review,ColinkaMalinka,https://disk.yandex.by/d/PdcjuyhLCYgIFw,2021-08-26 11:12:00,NaN,NaN
4611,CreATivE8536#5039,In review,isavazh,Validator: isavazh<br /><br />https://www.drop...,2021-08-27 15:47:00,NaN,NaN
4418,Danil Ushakov#5735,In review,ushakov,https://drive.google.com/file/d/19QPpwKpB7E999...,2021-08-26 11:37:00,NaN,NaN
4478,EdwardMorra#5755,In review,EdwardMorra,I've uploaded the file to your portal but sinc...,2021-08-26 14:37:00,NaN,NaN


In [128]:
!mkdir -p portal-review

In [129]:
sf5match.to_csv('portal-review/capture_submit_slog.csv')